# Automatic finding of thresholds of already calibrated data

### ... on NERSC!

In [ ]:
%matplotlib inline

import glob
import os
from collections.abc import Callable, Sequence, Iterator, Mapping
from abc import ABC, abstractmethod
from typing import Any

import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.axes import Axes

import numpy as np
import awkward as ak

from lgdo import lh5
from lgdo.lh5.exceptions import LH5DecodeError
from legendmeta import LegendMetadata
from dspeed.processors import get_multi_local_extrema

from sipm_autocalib import *

plt.rcParams["figure.figsize"] = (10, 4)

In [ ]:
# metadata dir has to be declared save git directory

base_dir = "/global/cfs/cdirs/m2676/data/lngs/l200/public/prodenv/prod-blind/tmp-auto"
dsp_dir = os.path.join(base_dir, "generated/tier/dsp/phy/p16/r000")
dsp_files = glob.glob(dsp_dir+"/l200-*-tier_dsp.lh5")
dsp_files.sort()
assert len(dsp_files) > 0
dsp_files = dsp_files[:50]
lmeta  = LegendMetadata(os.path.join(base_dir, "inputs"))
chmap = lmeta.channelmap(get_timestamp_from_filename(dsp_files[0]))
chmap_sipm = chmap.map("system", unique=False).spms
#requires recent legend-datasets
raw_keys = chmap_sipm.map("analysis.usability", unique=False).on.map("daq.rawid").keys()
onlythese = ["S037", "S050", "S051", "S054", "S080", "S083", "S095"]
if len(onlythese) > 0:
    raw_keys = [chmap[k].daq.rawid for k in onlythese]

In [ ]:
energies = get_energies(dsp_files, raw_keys, chmap, take_pulser_from_normal=True)

In [ ]:
calib, thresholds = read_override_file("../out/p15_ovr.yaml")
calib_p16, _ = read_override_file("../out/p16_ovr.yaml")
calib = calib | calib_p16 # merge re-calibrated SiPMs in


In [ ]:
calibrated_histos = get_calibrated_histograms(energies, calib, (0,5), 200)
thresholds = multi_valley_minima(calibrated_histos, mimimum_position=0.4) # compute new thresholds
_ = plot_all_pe_histograms_and_thresholds(calibrated_histos, thresholds, gridx=True)

In [ ]:
if False:
    output_override_file("../config/temp_overridex.yaml", calib, thresholds)
if True:
    update_override_file("../out/p16_ovr.yaml", "../out/newp16_ovr.yaml", None, thresholds)